# Setup

In [35]:
import os
import torch
import torch.nn as nn
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from pycocotools.coco import COCO
import torchvision.transforms as T
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random
import torchvision.transforms.functional as VF
from torch.nn import functional as F
import wandb
from itertools import cycle


In [40]:
COCO_PATH = "../../data/coco/"
IMG_DIR_TRAIN = os.path.join(COCO_PATH, "images/train2017")
IMG_DIR_VAL = os.path.join(COCO_PATH, "images/val2017")
ANN_FILE_TRAIN = os.path.join(COCO_PATH, "annotations/instances_train2017.json")
ANN_FILE_VAL = os.path.join(COCO_PATH, "annotations/instances_val2017.json")

GRID_ROWS = 6
GRID_COLS = 6
MAX_BOXES_PER_CELL = 2
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

DS_PERCENT = (0.1, 0.1, 0.1)

# Functions

## Data

In [18]:
class CustomTransform:
    def __init__(self, size=IMAGE_SIZE):
        self.transform = T.Compose([
            T.Resize(size),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __call__(self, image, target):
        return self.transform(image), target

In [19]:
class CocoBBoxDataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None):
        super().__init__(img_folder, ann_file)
        self.coco = COCO(ann_file)
        self._transforms = transforms
        
    def __getitem__(self, idx):
        img, target = super().__getitem__(idx)
        orig_w, orig_h = img.size
        
        anns = []
        for obj in target:
            if obj['category_id'] == 1:  # Only humans
                anns.append(obj)
                
        if self._transforms:
            img, _ = self._transforms(img, None)
            
        # Create target tensor [GRID_ROWS, GRID_COLS, MAX_BOXES_PER_CELL, 5]
        target_tensor = torch.zeros(GRID_ROWS, GRID_COLS, MAX_BOXES_PER_CELL, 5)
        
        for ann in anns:
            bbox = ann['bbox']  # [x, y, w, h]
            # Scale to image size
            x = bbox[0] * (IMAGE_SIZE[0] / orig_w)
            y = bbox[1] * (IMAGE_SIZE[1] / orig_h)
            w = bbox[2] * (IMAGE_SIZE[0] / orig_w)
            h = bbox[3] * (IMAGE_SIZE[1] / orig_h)
            
            # Convert to center coordinates
            x_center = x + w/2
            y_center = y + h/2
            
            # Find grid cell
            cell_i = min(int(y_center * GRID_ROWS / IMAGE_SIZE[1]), GRID_ROWS-1)
            cell_j = min(int(x_center * GRID_COLS / IMAGE_SIZE[0]), GRID_COLS-1)
            
            # Normalize to cell
            cell_w = IMAGE_SIZE[0] / GRID_COLS
            cell_h = IMAGE_SIZE[1] / GRID_ROWS
            x_cell = (x_center - cell_j * cell_w) / cell_w
            y_cell = (y_center - cell_i * cell_h) / cell_h
            w_norm = w / IMAGE_SIZE[0]
            h_norm = h / IMAGE_SIZE[1]
            
            # Find empty slot
            for k in range(MAX_BOXES_PER_CELL):
                if target_tensor[cell_i, cell_j, k, 4] == 0:  # Check if slot is empty
                    target_tensor[cell_i, cell_j, k] = torch.tensor([x_cell, y_cell, w_norm, h_norm, 1.0])
                    break
                    
        return img, target_tensor
    

## Model

In [20]:
MODEL_NAME = "BBoxModelSimple"

class BBoxModel(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d((GRID_ROWS, GRID_COLS))
        self.conv_head = nn.Conv2d(512, MAX_BOXES_PER_CELL * 5, kernel_size=1)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.avgpool(x)
        x = self.conv_head(x)
        x = x.permute(0, 2, 3, 1)
        x = x.view(x.size(0), GRID_ROWS, GRID_COLS, MAX_BOXES_PER_CELL, 5)
        return x


## Loss

In [21]:
CRITERION_NAME = "mse+bce"

def bbox_loss(pred, target, lambda_coord=5.0):
    # Extract components
    pred_xy = pred[..., :2]
    pred_wh = pred[..., 2:4]
    pred_conf = pred[..., 4]
    
    target_xy = target[..., :2]
    target_wh = target[..., 2:4]
    target_conf = target[..., 4]
    
    # Coordinate loss (only for boxes with objects)
    obj_mask = target_conf == 1
    if obj_mask.sum() > 0:
        xy_loss = nn.functional.mse_loss(
            pred_xy[obj_mask], 
            target_xy[obj_mask],
            reduction='sum'
        )
        wh_loss = nn.functional.mse_loss(
            pred_wh[obj_mask],
            target_wh[obj_mask],
            reduction='sum'
        )
        coord_loss = (xy_loss + wh_loss) * lambda_coord
    else:
        coord_loss = torch.tensor(0.0, device=pred.device)
    

    # Confidence loss (BCE)
    conf_loss = nn.functional.binary_cross_entropy_with_logits(
        pred_conf, target_conf, reduction='sum'
    )


    total_loss = (conf_loss + coord_loss) / BATCH_SIZE
    return total_loss

## Training

In [22]:
def train_one_epoch(model, dataloader, optimizer, criterion, verbose_tqdm=False):
    model.train()
    total_loss = 0.0

    dl = tqdm(dataloader, desc="Training") if verbose_tqdm else dataloader    
    for imgs, keypoints in dl:
        imgs = imgs.to(device)
        keypoints = keypoints.to(device)
        
        optimizer.zero_grad()
        
        preds = model(imgs)
        loss = criterion(preds, keypoints)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, verbose_tqdm=False):
    model.eval()
    total_loss = 0.0

    dl = tqdm(dataloader, desc="Evaluating") if verbose_tqdm else dataloader
    with torch.no_grad():
        for imgs, keypoints in dl:
            imgs = imgs.to(device)
            keypoints = keypoints.to(device)
            
            preds = model(imgs)
            loss = criterion(preds, keypoints)
            
            total_loss += loss.item()
    
    return total_loss / len(dataloader)



## Visualization

In [23]:
def visualize_bbox(img_tensor, target_bbox, pred_bbox=None, conf_threshold=0.5):
    img = img_tensor.cpu().permute(1, 2, 0).numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(img)
    
    # Draw grid
    for i in range(1, GRID_ROWS):
        ax.axhline(y=i * IMAGE_SIZE[1] / GRID_ROWS, color='r', linestyle='-', alpha=0.3)
    for j in range(1, GRID_COLS):
        ax.axvline(x=j * IMAGE_SIZE[0] / GRID_COLS, color='r', linestyle='-', alpha=0.3)
    
    # Draw ground truth boxes
    for i in range(GRID_ROWS):
        for j in range(GRID_COLS):
            for k in range(MAX_BOXES_PER_CELL):
                if target_bbox[i, j, k, 4] == 1:
                    x_cell, y_cell, w_norm, h_norm = target_bbox[i, j, k, :4]
                    
                    cell_w = IMAGE_SIZE[0] / GRID_COLS
                    cell_h = IMAGE_SIZE[1] / GRID_ROWS
                    
                    x_center = j * cell_w + x_cell * cell_w
                    y_center = i * cell_h + y_cell * cell_h
                    width = w_norm * IMAGE_SIZE[0]
                    height = h_norm * IMAGE_SIZE[1]
                    
                    x = x_center - width / 2
                    y = y_center - height / 2
                    
                    rect = patches.Rectangle(
                        (x, y), width, height, 
                        linewidth=2, edgecolor='g', facecolor='none'
                    )
                    ax.add_patch(rect)
    
    # Draw predicted boxes
    if pred_bbox is not None:
        pred_conf = torch.sigmoid(pred_bbox[..., 4])
        pred_xy = torch.sigmoid(pred_bbox[..., :2])
        pred_wh = torch.sigmoid(pred_bbox[..., 2:4])
        
        for i in range(GRID_ROWS):
            for j in range(GRID_COLS):
                for k in range(MAX_BOXES_PER_CELL):
                    if pred_conf[i, j, k] > conf_threshold:
                        x_cell, y_cell = pred_xy[i, j, k]
                        w_norm, h_norm = pred_wh[i, j, k]
                        
                        cell_w = IMAGE_SIZE[0] / GRID_COLS
                        cell_h = IMAGE_SIZE[1] / GRID_ROWS
                        
                        x_center = j * cell_w + x_cell * cell_w
                        y_center = i * cell_h + y_cell * cell_h
                        width = w_norm * IMAGE_SIZE[0]
                        height = h_norm * IMAGE_SIZE[1]
                        
                        x = x_center - width / 2
                        y = y_center - height / 2
                        
                        rect = patches.Rectangle(
                            (x, y), width, height, 
                            linewidth=2, edgecolor='r', facecolor='none', linestyle='--'
                        )
                        ax.add_patch(rect)
    
    plt.show()

# Training

## Dataloader

In [50]:
from torch.utils.data import Subset

# Create datasets
train_dataset = CocoBBoxDataset(IMG_DIR_TRAIN, ANN_FILE_TRAIN, transforms=CustomTransform())
val_dataset = CocoBBoxDataset(IMG_DIR_VAL, ANN_FILE_VAL, transforms=CustomTransform())

# Split val_dataset into validation and test sets
val_size = len(val_dataset)
test_split = 0.5  # 50% for validation, 50% for test (adjust as needed)
val_indices = list(range(val_size))
split = int(np.floor(test_split * val_size))
np.random.shuffle(val_indices)
val_idx, test_idx = val_indices[split:], val_indices[:split]


val_subset = Subset(val_dataset, val_idx)
test_subset = Subset(val_dataset, test_idx)

# Apply DS_PERCENT to subsample the datasets
train_percent, val_percent, test_percent = DS_PERCENT

if train_percent < 1.0:
    train_len = int(len(train_dataset) * train_percent)
    train_indices = np.random.choice(len(train_dataset), train_len, replace=False)
    train_indices = [int(i) for i in train_indices]
    train_dataset = Subset(train_dataset, train_indices)

if val_percent < 1.0:
    val_len = int(len(val_subset) * val_percent)
    val_indices_sub = np.random.choice(len(val_subset), val_len, replace=False)
    val_indices_sub = [int(i) for i in val_indices_sub]
    val_subset = Subset(val_subset, val_indices_sub)

if test_percent < 1.0:
    test_len = int(len(test_subset) * test_percent)
    test_indices_sub = np.random.choice(len(test_subset), test_len, replace=False)
    test_indices_sub = [int(i) for i in test_indices_sub]
    test_subset = Subset(test_subset, test_indices_sub)


# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)


loading annotations into memory...
Done (t=12.13s)
creating index...
index created!
loading annotations into memory...
Done (t=12.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!


In [51]:
print("Train dataset length:", len(train_dataset))
print("Validation dataset length:", len(val_subset))
print("Test dataset length:", len(test_subset))

Train dataset length: 11828
Validation dataset length: 250
Test dataset length: 250


## GT visualization

In [ ]:
# Draw a random validation image with only ground truth boxes
idx = random.randint(0, len(val_dataset) - 1)
img, target = val_dataset[idx]
visualize_bbox(img, target)

## Training Loop

In [53]:
EPOCHS = 100
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4

In [54]:
# Initialize model
model = BBoxModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

criterion = bbox_loss

In [55]:
wandb_config = {
    "epochs": EPOCHS,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "batch_size": BATCH_SIZE,
    "train_size": len(train_dataset),
    "val_size": len(val_subset),
    "test_size": len(test_subset),
    "grid_rows": GRID_ROWS,
    "grid_cols": GRID_COLS,
    "max_boxes_per_cell": MAX_BOXES_PER_CELL,
    "device": device,
    "image_size": IMAGE_SIZE,    
    "criterion": CRITERION_NAME,
    "optimizer": "Adam",
    "model_name": MODEL_NAME,
}

wandb.init(
    entity="fejowo5522-",
    project="NN_Project",
    config=wandb_config,
    group="BBoxDetection"
)


In [56]:
verbose_tqdm = True

early_stopping = True
patience = 10
best_val_loss = float('inf')
epochs_no_improve = 0

train_losses = []
val_losses = []

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    # Train
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, verbose_tqdm=verbose_tqdm)
    train_losses.append(train_loss)
    
    # Validate
    val_loss = evaluate(model, val_loader, criterion, verbose_tqdm=verbose_tqdm)
    val_losses.append(val_loss)
    
    # Log to wandb
    wandb.log({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'val_loss': val_loss
    })
    
    # Early stopping
    if early_stopping:
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/370 [00:00<?, ?it/s]

In [ ]:
# Load best model
model.load_state_dict(torch.load("best_model.pth"))

# Evaluate on test set
model.eval()
test_loss = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}")

# Log test results
wandb.log({'test_loss': test_loss})
wandb.finish()

# Save final model
torch.save(model.state_dict(), "bbox_model.pth")
print("Model saved to bbox_model.pth")


## Result evaluation and visualization

In [ ]:
model.eval()
img, target = next(iter(val_loader))
with torch.no_grad():
    pred = model(img.to(device))[0].cpu()
    
visualize_bbox(img[0], target[0], pred)